In [ ]:
import os
import pandas as pd
import glob

import sys; sys.path.insert(0, "./utils/")
from data_utils import get_data_location, get_clinical_df, get_bluepoints_df

DATA_PATH = get_data_location()
print("Infered Data path:", DATA_PATH)

In [ ]:
clinical_data = get_clinical_df()

# As datetime, format is DD-MM-YYYY
clinical_data.adm_date = pd.to_datetime(clinical_data.adm_date, format="%d-%m-%Y")
# Rename Video ID to Patient ID
clinical_data.rename(columns={"Video ID": "Patient ID"}, inplace=True)

# Add bluepoint data
bp = get_bluepoints_df()

# Filter patients that don't have video data
clinical_id = clinical_data["Patient ID"]
bp_id = bp["Patient ID"]

# Filter all clinical id that are not in the bluepoints
clinical_data =  clinical_data[clinical_id.isin(bp_id)].reset_index(drop=True)

In [ ]:
clinical_data

In [ ]:
age_col = "pat_birth_year"
gender_col = "pat_sex"
covid_col = "clin_diagn#COVID19_pneumonia"

In [ ]:
# Number of male vs female
clinical_data[gender_col].value_counts()

In [ ]:
# Number of patients with COVID-19
clinical_data[covid_col].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
# Transform birth year to age
clinical_data["Age"] = 2023 - clinical_data[age_col]

# Age distribution
clinical_data["Age"].hist()
mean_age = int(clinical_data["Age"].mean())
plt.title(f"Patient age distribution)")
plt.xlabel("Age")
plt.ylabel("Number of patients")

In [ ]:
# Plot admission adm_date histogram
clinical_data["adm_date"].hist()
plt.title("Admission date histogram")
plt.xlabel("Admission date")
plt.ylabel("Number of patients")

# Rotate x-axis labels
plt.xticks(rotation=90)

In [ ]:
clinical_data["COVID-19"] = clinical_data["clin_diagn#COVID19_pneumonia"].copy()
sns.set(font_scale=1.5)
# Plot age distribution and admission date
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(data=clinical_data, x="Age", ax=ax[0], hue="COVID-19")
ax[0].set_title("Age distribution")
ax[0].set_xlabel("Age")
ax[0].set_ylabel("Number of patients")

# Plot admission date. One bar per month
# Group by month
clinical_data["adm_date_month"] = clinical_data["adm_date"].dt.to_period("M")
# Transform to Month name + Year
clinical_data["adm_date_month"] = clinical_data["adm_date_month"].dt.strftime("%b %Y")

sns.histplot(data=clinical_data, x="adm_date_month", ax=ax[1], hue="COVID-19")
ax[1].set_title("Admission date distribution")
ax[1].set_xlabel("Admission month")
ax[1].set_ylabel("Number of patients")
# Change y-axis to integer
ax[1].set_yticks(range(0, 20, 2))


# Save figure
#fig.savefig("/Users/matthiasrichter/Library/CloudStorage/OneDrive-Personal/Studium/Masterthesis/Thesis/Figures/Methods/age_and_adm_date.png")

plt.show()

In [ ]:
# Adm date min and max
print("Admission date min:", clinical_data["adm_date"].min())
print("Admission date max:", clinical_data["adm_date"].max())

In [ ]:
# Total number of videos
bp["video_name"].nunique()

In [ ]:
bp.groupby("Patient ID").size().value_counts()

In [ ]:
pixel_count_path = os.path.join(DATA_PATH,"cropped_videos_segmented/post_processing")

# List all files & directories
files = os.listdir(pixel_count_path)

# keep only .csv files
csvs = [x for x in files if x.endswith(".gif_pixel_counts.csv")]

# Read all csvs into one dataframe
df = pd.concat((pd.read_csv(os.path.join(pixel_count_path, f)) for f in csvs))

In [ ]:
df